<a href="https://colab.research.google.com/github/infiniteoverflow/Distribute-Strategy/blob/master/Distribute_strategy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**We install tensorflow with gpu support**

In [1]:
pip install tensorflow-gpu

     |████████████████████████████████| 320.4MB 46kB/s 


In [2]:
import os
import time
import tensorflow as tf
import numpy as np

print(tf.version)

<module 'tensorflow._api.v2.version' from '/usr/local/lib/python3.6/dist-packages/tensorflow/_api/v2/version/__init__.py'>


In [3]:
tf.test.gpu_device_name()

'/device:GPU:0'

In [6]:
from tensorflow.python.client import device_lib

def get_available_gpus():
  local_device_protos = device_lib.list_local_devices()
  return [x.name for x in local_device_protos  if x.device_type=='GPU']

get_available_gpus()

['/device:GPU:0']

**Importing the dataset and preprocessing it**

In [8]:
LABEL_DIMENSIONS = 10

(train_images,train_labels),(test_images,test_labels) = tf.keras.datasets.fashion_mnist.load_data()

TRAINING_SIZE = len(train_images)
TEST_SIZE = len(test_images)

train_images = np.asarray(train_images,dtype=np.float32) / 255
test_images = np.asarray(test_images,dtype=np.float32) / 255

train_images = train_images.reshape((TRAINING_SIZE,28,28,1))
test_images = test_images.reshape((TEST_SIZE,28,28,1))

train_labels = tf.keras.utils.to_categorical(train_labels,LABEL_DIMENSIONS)
test_labels = tf.keras.utils.to_categorical(test_labels,LABEL_DIMENSIONS)

train_labels = train_labels.astype(np.float32)
test_labels = test_labels.astype(np.float32)

In [9]:
print(train_images.shape , train_labels.shape)
print(test_images.shape , test_labels.shape)

(60000, 28, 28, 1) (60000, 10)
(10000, 28, 28, 1) (10000, 10)


**Now we build a CNN model using the functional API**

In [11]:
inputs = tf.keras.Input(shape=(28,28,1))

x = tf.keras.layers.Conv2D(filters=32,kernel_size=(3,3),activation='relu')
x = x(inputs)

x = tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=2)(x)

x = tf.keras.layers.Conv2D(filters=64,kernel_size=(3,3),activation='relu')(x)

x = tf.keras.layers.MaxPooling2D(pool_size=(2,2),strides=2)(x)

x = tf.keras.layers.Conv2D(filters=256,kernel_size=(3,3),activation='relu')(x)

x = tf.keras.layers.Flatten()(x)

x = tf.keras.layers.Dense(64,activation='relu')(x)

x = tf.keras.layers.Dropout(0.2)(x)

x = tf.keras.layers.Dense(128,activation='relu')(x)

predictions = tf.keras.layers.Dense(LABEL_DIMENSIONS,activation='softmax')(x)

In [12]:
model = tf.keras.Model(inputs=inputs,outputs=predictions)
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 28, 28, 1)]       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 3, 3, 256)         147712    
_________________________________________________________________
flatten (Flatten)            (None, 2304)             

**Compiling the model**

In [13]:
optimizer = tf.keras.optimizers.SGD()
model.compile(loss='categorical_crossentropy',
              optimizer = optimizer,
              metrics = ['accuracy'])

**Defining the strategy**

In [14]:
strategy = tf.distribute.MirroredStrategy()
config = tf.estimator.RunConfig(train_distribute=strategy)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0',)
INFO:tensorflow:Initializing RunConfig with distribution strategies.
INFO:tensorflow:Not using Distribute Coordinator.


**Converting Keras to Estimator**

In [15]:
estimator = tf.keras.estimator.model_to_estimator(model,config=config)

INFO:tensorflow:Using the Keras model provided.
Instructions for updating:
Simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/tmpjktxy4sx', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': <tensorflow.python.distribute.mirrored_strategy.MirroredStrategy object at 0x7fd6b993a898>, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 

**Input function for estimator**

In [16]:
def input_func(images,labels,epochs,batch_size):
  dataset = tf.data.Dataset.from_tensor_slices((images,labels))
  SHUFFLE_SIZE = 5000

  dataset = dataset.shuffle(SHUFFLE_SIZE).repeat(epochs).batch(batch_size)
  dataset = dataset.prefetch(None)

  return dataset

**Training**

In [17]:
BATCH_SIZE = 512
EPOCHS = 50

estimator_train_result = estimator.train(
    input_fn=lambda: input_func(train_images,train_labels,EPOCHS,BATCH_SIZE)
)

print(estimator_train_result)

Instructions for updating:
Use `tf.data.Iterator.get_next_as_optional()` instead.
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /replica:0/task:0/device:CPU:0 then broadcast to ('/replica:0/task:0/devic

In [19]:
estimator.evaluate(lambda: input_func(test_images,test_labels,1,BATCH_SIZE))

INFO:tensorflow:Calling model_fn.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-08-25T13:35:09Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmpjktxy4sx/model.ckpt-5860
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 0.43467s
INFO:tensorflow:Finished evaluation at 2020-08-25-13:35:09
INFO:tensorflow:Saving dict for global step 5860: accuracy = 0.8313, global_step = 5860, loss = 0.4618309
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 5860: /tmp/tmpjktxy4sx/model.ckpt-5860


{'accuracy': 0.8313, 'global_step': 5860, 'loss': 0.4618309}